In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
pd.set_option('mode.chained_assignment',  None) # 경고를 끈다
pd.set_option('display.max_columns', None) # 모든 열 출력
plt.rcParams['axes.unicode_minus'] = False

In [2]:
taas = pd.read_csv('data/taas/taas_only.csv', encoding='cp949')

### df 생성(지역 합치기)

In [3]:
taas.head(1)

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,x,y
0,2019010100100002,2019년 1월 1일 00시,화요일,서울특별시 구로구 고척동,경상사고,0,0,1,0,차대차 - 추돌,안전운전불이행,포장 - 건조,맑음,단일로 - 기타,이륜,남,23세,상해없음,승용,남,71세,경상,126.867286,37.499889


In [4]:
df = taas.copy()

### 사고번호 유지(PK)

### 사고일시 전처리

In [5]:
# 사고일시 형식 변경
df['사고일시'] = pd.to_datetime(df['사고일시'], format='%Y년 %m월 %d일 %H시')
df['사고일시'] = df['사고일시'].dt.strftime('%Y%m%d%H%M')

### 요일 유지 : 7가지

In [6]:
df['요일'].value_counts()

금요일    9771
토요일    9200
목요일    8766
화요일    8703
수요일    8690
월요일    8419
일요일    7866
Name: 요일, dtype: int64

### 시군구 유지

In [7]:
df1 = df.copy()

In [8]:
df1[df1['시군구'].str.contains('미분류')]

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,x,y
7098,2019110800100396,201911081400,금요일,미분류 미분류 미분류,중상사고,0,1,0,0,차대차 - 측면충돌,신호위반,포장 - 건조,맑음,교차로 - 교차로안,승합,남,56세,상해없음,이륜,남,44세,중상,127.068553,37.593779


In [9]:
# 가입 없이 좌표->주소 변환
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

address = geocoding_reverse('37.593779, 127.068553')
print(address)

379, 한천로, 이문동, 휘경1동, 동대문구, 서울, 02403, 대한민국


In [10]:
df1.iloc[7098,3] = '서울특별시 동대문구 휘경1동'

In [11]:
df1[df1['시군구'].str.contains('미분류')]

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,x,y


### 사고내용 전처리(drop)

In [12]:
# 사고내용 column drop
df1.drop('사고내용', axis=1, inplace=True)

### 사고유형 전처리 : 3가지

In [13]:
df1["사고유형"].value_counts()

차대차 - 측면충돌           25933
차대차 - 기타             19248
차대차 - 추돌              6134
차대차 - 정면충돌            2502
차대사람 - 횡단중            2431
차대사람 - 기타             1444
차대차 - 후진중충돌            801
차량단독 - 기타              728
차량단독 - 전도전복 - 전도       679
차대사람 - 보도통행중           454
차량단독 - 공작물충돌           428
차대사람 - 차도통행중           420
차대사람 - 길가장자리구역통행중      119
차량단독 - 도로외이탈 - 추락       36
차량단독 - 도로외이탈 - 기타       27
차량단독 - 전도전복 - 전복        22
차량단독 - 주/정차차량 충돌         8
철길건널목 - 철길건널목            1
Name: 사고유형, dtype: int64

In [14]:
# 사고유형 중에 "철길건널목 - 철길건널목" 삭제
df1 = df1[df1['사고유형'] != "철길건널목 - 철길건널목"]

In [15]:
df1['사고유형'] = df1['사고유형'].apply(lambda x: x.split(" - ")[0])

In [16]:
df1['사고유형'].value_counts()

차대차     54618
차대사람     4868
차량단독     1928
Name: 사고유형, dtype: int64

### 법규위반 유지 : 11가지

In [17]:
df1['법규위반'].value_counts()

안전운전불이행      29395
신호위반         11240
안전거리미확보       6390
교차로운행방법위반     4287
중앙선침범         3053
기타            2388
직진우회전진행방해     2251
차로위반          1089
보행자보호의무위반      616
불법유턴           572
과속             133
Name: 법규위반, dtype: int64

### 노면상태 전처리 : 7가지

In [18]:
df1['노면상태'] = df1['노면상태'].apply(lambda x: x.split(" - ")[1])

In [19]:
df1['노면상태'].value_counts()

건조       54625
젖음/습기     6044
기타         595
서리/결빙      110
적설          30
침수           7
해빙           3
Name: 노면상태, dtype: int64

### 기상상태 유지 : 6가지

In [20]:
df1['기상상태'].value_counts()

맑음    53907
비      4577
흐림     2271
기타      546
눈       103
안개       10
Name: 기상상태, dtype: int64

### 도로형태 유지 : 11가지

In [21]:
df1['도로형태'].value_counts()

단일로 - 기타           25258
교차로 - 교차로안         22414
교차로 - 교차로부근         8943
기타 - 기타             2276
교차로 - 교차로횡단보도내      1444
단일로 - 지하차도(도로)내      551
단일로 - 교량             242
단일로 - 고가도로위          122
단일로 - 터널              94
주차장 - 주차장             58
미분류 - 미분류             12
Name: 도로형태, dtype: int64

### 가해운전자 차종 : 유지

In [22]:
df1['가해운전자 차종'].value_counts()

이륜             28600
승용             26526
화물              3183
승합              1295
기타불명             836
자전거              523
건설기계             164
개인형이동수단(PM)      108
원동기               82
특수                80
농기계               12
사륜오토바이(ATV)        5
Name: 가해운전자 차종, dtype: int64

### 가해운전자 성별 유지 : 3가지

In [23]:
df1['가해운전자 성별'].value_counts()

남       52250
여        8239
기타불명      925
Name: 가해운전자 성별, dtype: int64

### 가해운전자 연령 : 5가지
+ 청소년(13~18세)
+ 청년(19~29세)
+ 중년(30~49세)
+ 장년(50~64세)
+ 노년(65세 이상)

In [24]:
df1['가해운전자 연령'] = df1['가해운전자 연령'].replace('90세 이상', '90세')

In [25]:
# df1의 '가해운전자 연령' column에서 '세' 문자열 제거
df1['가해운전자 연령'] = df1['가해운전자 연령'].apply(lambda x: x.rstrip('세'))

# '가해운전자 연령' column의 모든 값이 숫자로 변환 가능한지 확인
age_col = pd.to_numeric(df1['가해운전자 연령'], errors='coerce')

# 변환 가능하지 않은 값은 NaN 값으로 변환되어 mean 함수 결과는 NaN 값이 나옴
mean_age = np.round(age_col.mean(), 0)

# '미분류' 값을 해당 column의 숫자평균값으로 변경
df1['가해운전자 연령'] = df1['가해운전자 연령'].replace('미분류', int(mean_age))

In [26]:
df1['가해운전자 연령'] = df1['가해운전자 연령'].astype(int)

In [27]:
df1['가해운전자 연령'] = df1['가해운전자 연령'].apply(lambda x: '청소년' if 13<=x<=18 else ('청년' if 19<=x<=29 else ('중년' if 30<=x<=49 else ('장년' if 50<=x<=64 else '노년'))))

In [28]:
df1['가해운전자 연령'].value_counts()

중년     21583
장년     16788
청년     12734
노년      7420
청소년     2889
Name: 가해운전자 연령, dtype: int64

### 가해운전자 상해정도, 피해운전자 차종, 피해운전자 성별, 피해운전자 연령, 피해운전자 상해정도 전처리(drop)

In [29]:
# column drop
df1.drop(['가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도'], axis=1, inplace=True)

In [30]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61414 entries, 0 to 61414
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   사고번호      61414 non-null  int64  
 1   사고일시      61414 non-null  object 
 2   요일        61414 non-null  object 
 3   시군구       61414 non-null  object 
 4   사망자수      61414 non-null  int64  
 5   중상자수      61414 non-null  int64  
 6   경상자수      61414 non-null  int64  
 7   부상신고자수    61414 non-null  int64  
 8   사고유형      61414 non-null  object 
 9   법규위반      61414 non-null  object 
 10  노면상태      61414 non-null  object 
 11  기상상태      61414 non-null  object 
 12  도로형태      61414 non-null  object 
 13  가해운전자 차종  61414 non-null  object 
 14  가해운전자 성별  61414 non-null  object 
 15  가해운전자 연령  61414 non-null  object 
 16  x         61414 non-null  float64
 17  y         61414 non-null  float64
dtypes: float64(2), int64(5), object(11)
memory usage: 8.9+ MB


### EPDO(대물피해환산계수) = 사망자수 x 12 + 중상자수 x 5 + 경상자수 x 3 + 부상신고자수

In [31]:
df1['EPDO'] = 12 * df1['사망자수'] + 5 * df1['중상자수'] + 3 * df1['경상자수'] + df1['부상신고자수']

### 사망자수, 중상자수, 경상자수, 부상신고자수 전처리(drop)

In [32]:
# column drop
df1.drop(['사망자수', '중상자수', '경상자수', '부상신고자수'], axis=1, inplace=True)

In [33]:
taas_1 = df1.copy()

## TAAS 전처리 완료 dataframe

In [34]:
taas_1

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO
0,2019010100100002,201901010000,화요일,서울특별시 구로구 고척동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,126.867286,37.499889,3
1,2019010100100141,201901011100,화요일,서울특별시 서초구 서초동,차대차,안전거리미확보,건조,맑음,단일로 - 기타,승용,남,중년,127.017439,37.481949,3
2,2019010100100170,201901011300,화요일,서울특별시 서대문구 북아현동,차대사람,보행자보호의무위반,건조,맑음,단일로 - 기타,이륜,남,청년,126.956285,37.558028,3
3,2019010100100236,201901011700,화요일,서울특별시 중랑구 상봉동,차대차,신호위반,건조,맑음,교차로 - 교차로안,이륜,남,청년,127.085745,37.596356,3
4,2019010100100249,201901011800,화요일,서울특별시 구로구 천왕동,차대차,안전운전불이행,젖음/습기,맑음,교차로 - 교차로안,이륜,남,청년,126.842479,37.484115,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61410,2021123000100493,202112301900,목요일,울산광역시 남구 삼산동,차량단독,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,중년,129.351972,35.538223,1
61411,2021123100100141,202112311100,금요일,울산광역시 동구 일산동,차대차,안전운전불이행,건조,맑음,기타 - 기타,이륜,남,청소년,129.428386,35.498666,1
61412,2021123100100408,202112311700,금요일,울산광역시 중구 복산동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,승용,남,노년,129.329986,35.563774,5
61413,2021123100100409,202112311700,금요일,울산광역시 북구 염포동,차대차,차로위반,건조,맑음,단일로 - 기타,승합,남,중년,129.389056,35.535538,5


In [35]:
# 합쳐진 taas 내보내기
taas_1.to_csv('data/taas/taas_preprocessed.csv', index=False, encoding = 'cp949')